Imports and check if any CUDA device is available.

In [ ]:
import torchtt as tntt
import torch as tn
import datetime

print('CUDA available:',tn.cuda.is_available())
print('Device name: ' + tn.cuda.get_device_name())

Define a function to test. It performs 2 matrix vector products in TT-format and a rank rounding.
The return result is a scalar.

In [ ]:
def f(x,A,y):
    """
    fonction that performs operations of tensors in TT.

    Args:
        x (tnt.TT): input TT tensor
        A (tnt.TT): input TT matrix
        y (tnt.TT): input TT tensor

    Returns:
        torch.tensor: result
    """
    z = A @ y + A @ y # operatio that grows the rank
    z = z.round(1e-12) # rank rounding (contains QR and SVD decomposition)
    z += z+x # some other operation
    return tntt.dot(x,z) # contract the tensor

Generate random tensors in the TT-format (on the CPU).

In [ ]:
x = tntt.random([200,300,400,500],[1,8,8,8,1])
y = tntt.random([200,300,400,500],[1,8,8,8,1])
A = tntt.random([(200,200),(300,300),(400,400),(500,500)],[1,8,8,8,1])

Run the function f and report the time.

In [ ]:
tme_cpu = datetime.datetime.now()
f(x,A,y) 
tme_cpu = datetime.datetime.now() - tme_cpu
print('Time without CUDA: ',tme_cpu)

Move the defined tensors on GPU. Similarily to pytorch tensors one can use the function cuda() to return a copy of a TT instance on the GPU.
All the cores of the returned TT object are on the GPU.

In [ ]:
x = x.cuda()
y = y.cuda()
A = A.cuda()

The function is executed once without timing to "warm-up" the CUDA.

In [ ]:
f(x,A,y).cpu()

Run the function again. This time the runtime is reported. The return value is moved to CPU to assure blocking until all computations are done.

In [ ]:
tme_gpu = datetime.datetime.now()
f(x,A,y).cpu()
tme_gpu = datetime.datetime.now() - tme_gpu
print('Time with CUDA: ',tme_gpu)

The speedup is reported:

In [ ]:
print('Speedup: ',tme_cpu.total_seconds()/tme_gpu.total_seconds(),' times.')

A tensor can be copied to a differenct device using the to() method. Usage is similar to torch.tensor.to()

In [ ]:
dev = tn.cuda.current_device()
x_cuda = x.to(dev)
x_cpu = x_cuda.to(None)